# 3.0 ResNet Architecture
The ResNet architecture introduces the **Residual Learning** block, a fundamental improvement on deeper models. We will talk about the motivation for a deeper neural network, the core problem that hinders it, and how ResNet mitigates this problem. 

For this notebook, I'd like to quote my Multivariable Calculus professor, I-Shen Lai. Say hello to Professor Lai!

> "Haha... you can quote me if you want."
> I-Shen Lai, September 3rd 2025.

## 3.1 Deep Learning
Deep Learning models are made up of multiple sequential layers of other model, usually composed of the models we made in earlier notebooks. The addition of newer layers allow for more representation of complex features, allowing more information to be learned resulting in an even lower loss [[1]](#ref1).

However, there is a limit to the number of layers we can effectively add. In practice, when we reach tens of layers, the model faces a **Degradation Problem** that result in stagnant accuracy, accompanied by higher training errors [[1]](#ref). 

He et al. proposed that the layers themselves could not fit or approximate to the **identity function**, which is the function that simply return input itself [[1]](#ref).

So if we could somehow skip layers we deem useless or that it would just result in overfitting, then theoretically, we could improve our model, where adding additional layers would not worsen its performance.
> *Drum Rolls*
## 3.2 Residual Learning Block

## References
<a name="ref1">[1]</a> K. He, C. Zhang, S. Ren, and J. Sun, "Deep residual learning for image recognition," in Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2016, pp. 770-778.